In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [6]:
train_df = pd.read_csv("data_details/trial_promo_training.csv")
test_df = pd.read_csv("data_details/trial_promo_testing.csv")

In [7]:
train_df["age"] = preprocessing.scale(train_df["age"])
train_df["income"] = preprocessing.scale(train_df["income"])
train_df["avbal"] = preprocessing.scale(train_df["avbal"])
train_df["avtrans"] = preprocessing.scale(train_df["avtrans"])
train_df.head()

,F,M,married,widowed,single,divorced,age,children,construct,education,...,medicine,retired,secondary,tertiary,postgrad,professional,income,avbal,avtrans,decision
0,1.0,0.0,1.0,0.0,0.0,0.0,1.149005,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,-0.880655,-0.168665,0.440274,None
1,0.0,1.0,0.0,0.0,1.0,0.0,-1.215123,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,-0.880655,-0.168665,0.440274,None
2,1.0,0.0,1.0,0.0,0.0,0.0,1.196440,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,-1.448082,-0.284005,-0.635124,None
3,0.0,1.0,1.0,0.0,0.0,0.0,-0.810390,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,-0.880655,-0.168665,0.440274,A
4,0.0,1.0,0.0,0.0,1.0,0.0,-0.684916,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-0.430784,-0.260782,-0.716738,None


### Randomly shuffle the records since some of the decisions were clubbed after the over-sampling

In [10]:
train_df =  train_df.sample(frac=1).reset_index(drop=True)
train_df.head()

,F,M,married,widowed,single,divorced,age,children,construct,education,...,medicine,retired,secondary,tertiary,postgrad,professional,income,avbal,avtrans,decision
0,0.0,1.0,0.0,0.0,1.0,0.0,-0.642836,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-0.197810,0.104532,-0.983427,None
1,1.0,0.0,0.0,0.0,1.0,0.0,-0.796302,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,-0.074632,-0.738438,-0.900050,B
2,0.0,1.0,0.0,0.0,1.0,0.0,-0.717049,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.426730,0.211994,0.039693,None
3,0.0,1.0,0.0,0.0,1.0,0.0,-0.897691,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,-0.798521,0.184990,-0.149423,B
4,0.0,1.0,0.0,0.0,1.0,0.0,-0.767851,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.446665,0.368328,0.526002,B


In [12]:
X_train = train_df.loc[:, train_df.columns != "decision"]
y_train = train_df["decision"].replace("None", 0).replace("A", 1).replace("B", 2)
X_train.head()

,F,M,married,widowed,single,divorced,age,children,construct,education,...,manuf,medicine,retired,secondary,tertiary,postgrad,professional,income,avbal,avtrans
0,0.0,1.0,0.0,0.0,1.0,0.0,-0.642836,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.197810,0.104532,-0.983427
1,1.0,0.0,0.0,0.0,1.0,0.0,-0.796302,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.074632,-0.738438,-0.900050
2,0.0,1.0,0.0,0.0,1.0,0.0,-0.717049,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.426730,0.211994,0.039693
3,0.0,1.0,0.0,0.0,1.0,0.0,-0.897691,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.798521,0.184990,-0.149423
4,0.0,1.0,0.0,0.0,1.0,0.0,-0.767851,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.446665,0.368328,0.526002


In [17]:
y_train.head()

0    0
1    2
2    0
3    2
4    2
Name: decision, dtype: int64

In [18]:
test_df["age"] = preprocessing.scale(test_df["age"])
test_df["income"] = preprocessing.scale(test_df["income"])
test_df["avbal"] = preprocessing.scale(test_df["avbal"])
test_df["avtrans"] = preprocessing.scale(test_df["avtrans"])
test_df.head()

X_test = test_df.loc[:, test_df.columns != "decision"]
y_test = test_df["decision"].replace("None", 0).replace("A", 1).replace("B", 2)
X_test.head()

,F,M,married,widowed,single,divorced,age,children,construct,education,...,manuf,medicine,retired,secondary,tertiary,postgrad,professional,income,avbal,avtrans
0,1,0,1,0,0,0,1.694579,0,0,0,...,0,0,1,1,0,0,0,-1.055675,-1.348957,-0.814692
1,0,1,0,0,1,0,-1.051218,0,0,0,...,0,0,0,1,0,0,0,-1.009078,-0.330144,-0.691148
2,1,0,0,0,0,1,0.771610,2,0,0,...,0,0,0,0,1,0,0,0.222033,0.308487,-0.877786
3,1,0,0,0,1,0,-1.007801,0,0,0,...,0,0,0,0,1,0,0,0.487620,-0.646084,0.815205
4,1,0,0,0,0,1,0.340238,0,0,0,...,1,0,0,0,0,0,1,1.711120,-0.340193,1.348488


### Neural network

In [20]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X_train, y_train)


y_test_pred = clf.predict(X_test)
confusion_matrix(y_test, y_test_pred, labels=[0, 1, 2])

array([[109,  69,  29],
       [  1,  32,   0],
       [  3,   0,   7]], dtype=int64)